In [ ]:
import random
import numpy as np
import pandas as pd

from math import*

import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('../input/DataNew1.csv')
dadt = []
dadt.append(0)
for i in range(1,data.shape[0]):
    dadt.append((data.Alpha[i]-data.Alpha[i-1])/(data.Time[i]-data.Time[i-1]))

data['da-dt'] = dadt
df = data.drop(['Time'],axis=1)
df = df[df['da-dt'] != 0]
df = df.reset_index(drop=True)
df = df.dropna()
df.shape

In [ ]:
def fitness(i, pop):
    errorSum = []
    # unknowns
    m = pop.iloc[i, 0]
    n = pop.iloc[i, 1]
    a1 = pop.iloc[i, 2]
    a2 = pop.iloc[i, 3]
    e1 = pop.iloc[i, 4]
    e2 = pop.iloc[i, 5]

    for j in range(0, df.shape[0]):
        # given values
        T = df.iloc[j, 0]
        a = df.iloc[j, 1]
        d = df.iloc[j, 2]

        k1 = a1 * e ** (-e1 / T)
        k2 = a2 * e ** (-e2 / T)
        error = d - (k1 + k2 * a ** m) * (1 - a) ** n
        errorSum.append(error)
        errorSum = list(map(abs, errorSum))
    return 1 / sum(errorSum)
# Sample answer
# order: df = T-0, a-1, d-2; pop = m-0, n-1, a1-2, a2-3, e1-4, e2-5
#pp = initialPopulation(100, 6)
#fitness(0, pp)

In [ ]:
def No_use_initialPopulation(popsize, unknowns):
    pop = pd.DataFrame(np.random.randint(-2000, 16000, size=(popsize,unknowns)) * random.random(), columns=list('012345'))
    pop.columns = ['m', 'n', 'a1', 'a2', 'e1', 'e2']
    return pd.DataFrame(pop)
#initialPopulation(popsize=100, unknowns=6)

In [ ]:
def initialPopulation(popsize, unknowns):
    pop = pd.DataFrame(np.random.randint(0, 16000, size=(popsize,unknowns)) * random.random(), columns=list('012345'))
    pop.columns = ['m', 'n', 'a1', 'a2', 'e1', 'e2']
    pop.m = pop.m/700
    pop.n = pop.n/700
    pop.a1 = pop.a1/100000
    pop.a2 = pop.a2/120000000
    #pop.e1 = pop.e1*2
    #pop.e2 = pop.e2*2
    return pd.DataFrame(pop)
#initialPopulation(popsize=100, unknowns=6)

In [ ]:
def rankPopulation(population):
    dfr = population
    fitnessColumn = []
    for i in range(0, dfr.shape[0]):
        fitnessColumn.append(fitness(i,dfr))
    dfr['Fitness'] = fitnessColumn
    dfr.sort_values(by=['Fitness'], ascending=False, inplace=True)
    dfr = dfr.reset_index(drop=True)
    return dfr
#pp = initialPopulation(100, 6)
#rankPopulation(pp)

In [ ]:
# selection for meting = metingpool
def selection(rankedPop, eliteSize):
    dfs = rankedPop
    dfs['cum_sum'] = dfs.Fitness.cumsum()
    dfs['cum_perc'] = 100 * dfs.cum_sum / dfs.Fitness.sum()
    selectionResults = dfs.iloc[0:eliteSize, :]
    for i in range(0, dfs.shape[0] - eliteSize):
        pick = random.random()*100
        for i in range(0, dfs.shape[0]):
            if pick <= float(dfs.iloc[i, 5]):
                selectionResults = pd.concat([selectionResults, dfs.iloc[i:i + 1, :]], ignore_index=True)
                break
    return selectionResults.drop(['cum_sum', 'cum_perc', 'Fitness'], axis=1).values.tolist()

#pp = initialPopulation(100, 6)
#rp = rankPopulation(pp)
#selection(rp,20)

In [ ]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    for i in range(startGene, endGene):
        childP1.append(parent1[i])
    childP2 = [item for item in parent2 if item not in childP1]
    for i in range(0, len(parent1) - len(childP1)):
        childP1.append(childP2[i])
    child = childP1
    return child

In [ ]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0, eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool) - i - 1])
        children.append(child)
    return pd.DataFrame(children)

#pp = initialPopulation(100, 6)
#rp = rankPopulation(pp)
#mtp = selection(rp,20)
#breedPopulation(mtp, 20)

In [ ]:
def mutatePopulation(population, eliteSize):
    population = population.values.tolist()
    mutatedPop = []
    for i in range(0, eliteSize):
        mutatedPop.append(population[i])

    for j in range(0, len(population) - eliteSize):
        if j % 3 == 0:
            mutatedPop.append(
                list(np.array(population[0]) + np.random.randint(-1, 1, size=len(population[0])) * 5 * random.random()))
        elif j % 6 == 0:
            mutatedPop.append(list(np.array(population[0]) + np.random.randint(-1, 1, size=len(population[0]))))
        elif j % 5 ==0:
            mutatedPop.append((initialPopulation(len(population), len(population[0])).iloc[j]*np.random.random()).values.tolist())
        else:
            mutatedPop.append(initialPopulation(len(population), len(population[0])).iloc[j].values.tolist())
    return pd.DataFrame(mutatedPop)

# pp = initialPopulation(100, 6)
# rp = rankPopulation(pp)
# mtpl = selection(rp,20)
# bpp = breedPopulation(mtp, 20)
# mutatePopulation(bpp, 20)

In [ ]:
def nextGeneration(currentGen, eliteSize, unknowns):
    rankedPop = rankPopulation(currentGen)#.drop(['Fitness'], axis = 1) #fitness column required
    matingpool = selection(rankedPop,eliteSize)
    breedChildren = breedPopulation(matingpool, eliteSize)
    rankedChildren = rankPopulation(breedChildren).drop(['Fitness'], axis = 1)
    mutatedGen = mutatePopulation(rankedChildren, eliteSize)
    nextGeneration = rankPopulation(mutatedGen)#.drop(['Fitness'], axis = 1)#.values.tolist()
    #nextGeneration = children
    return nextGeneration

#pp = initialPopulation(100, 6)
#nextGeneration(currentGen=pp, eliteSize=20, unknowns=6)

In [ ]:
def geneticAlgorithm(unknowns, popSize, eliteSize, generations):
    pop = initialPopulation(popSize, unknowns)
    print(len(pop))
    print(rankPopulation(pop).shape)
    print("Initial error: " + str(1 / rankPopulation(pop).iloc[0, unknowns]))
    #fig, ex = plt.figure()
    progress = []
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, unknowns)
        progress.append(1 / rankPopulation(pop).iloc[0, unknowns])
        # print(len(pop))
        # print(rankPopulation(pop).shape)
        if i % 100 == 0:
            print(i)
            print("current error: " + str(1 / rankPopulation(pop).iloc[0, unknowns]))
        if i%200 == 0:
            print(str(rankPopulation(pop).iloc[0, :]))
        print(i)
        print(pop.iloc[0,:].values.tolist())
        print(1 / fitness(0,pop))
    plt.plot(progress)
    plt.suptitle('Current solution: [m, n, a1, a2, e1, e2] Error\n'+ str(np.round(pop.iloc[0,:][:-1],3).values.tolist())+"   "+ str(1/np.round(pop.iloc[0,:][-1],3)), fontsize=10) # np.round(a, decimals=0, out=None)
    #plt.text(2, 6, r'an equation: $E=mc^2$', fontsize=15)
    plt.ylabel('Error')
    plt.xlabel('Generation')
    plt.pause(0.05)
    plt.show()
    print("Final error: " + str(1 / rankPopulation(pop).iloc[0, unknowns]))
    bestSol = rankPopulation(pop).iloc[0:25, :]
    return bestSol

In [ ]:
ans = geneticAlgorithm(unknowns = 6, popSize=500, eliteSize=20, generations=2000)

In [ ]:
ans1 = ans.values.tolist()
ans2 = ans.values.tolist()
ans3 = []
ans3.append(ans1)
ans3.append(ans2)
ans4 = pd.DataFrame(ans3)
ans4

In [ ]:
1/fitness(0,ans4)

In [ ]:
gans1 = list([1.21,1.57, 0.03353,0.00003054,7161,8589])
gans2 = gans1
gans3 = []
gans3.append(gans1)
gans3.append(gans2)
gans4 = pd.DataFrame(gans3)
gans4
#m-0, n-1, a1-2, a2-3, e1-4, e2-5

In [ ]:
1/fitness(0,gans4)